In [1]:
!pip install pymongo
!pip install imblearn

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrad

In [4]:
import pymongo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import LeaveOneOut, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE, RandomOverSampler

def crear_dataframe_desde_mongodb():
    cadena_conexion = ('mongodb://{nom_usuario}:{password}@{host}:{port}')
    cadena_conexion = cadena_conexion.format(
        nom_usuario = 'PY01_c02',
        password = 'P4rd83XkXrTz',
        host = '5.189.129.12',
        port = 27017
    )
    client = pymongo.MongoClient(cadena_conexion)
    db = client.PY01
    header = [*db.PY01.find_one().keys()]
    lista_dataset = list()
    for doc in db.PY01.find():
        lista_dataset.append([*doc.values()])
 
    df = pd.DataFrame(data = lista_dataset, columns = header)
    return df

def generar_diccionario_meses():
    return {'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8, 'Setiembre': 9, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12}

def generar_diccionario_dias():
    return {'LUNES': 1, 'MONDAY': 1, 'MARTES': 2, 'TUESDAY': 2, 'MIERCOLES': 3, 'WEDNESDAY': 3, 'JUEVES': 4, 'THURSDAY': 4, 'VIERNES': 5, 'FRIDAY': 5, 'SABADO': 6, 'SATURDAY': 6, 'DOMINGO': 7, 'SUNDAY': 7}

def limpiar_dataset(df):
    dias = generar_diccionario_dias()
    meses = generar_diccionario_meses()
    for dia, cod in dias.items():
        df.loc[df.dia == dia, 'dia'] = cod
    df.dia = pd.to_numeric(df.dia)
    
    for mes, cod in meses.items():
        df.loc[df.mes == mes, 'mes'] = cod
    df.mes = pd.to_numeric(df.mes)
    
    return df

def generar_variables(df):
    nuevos_registros = list()
    for row in df.itertuples():
        cantidad = row.cantidad
        if cantidad > 1:
            cantidad -= 1
            for t in range(cantidad):
                nuevos_registros.append(row)

    df = df.append(nuevos_registros, ignore_index = True).drop(labels = ['cantidad'], axis = 1)

    df = pd.concat((df, pd.get_dummies(df.dia, prefix = 'dia')), axis = 1)
    dias_presentes = df.dia.unique()
    if 1 not in dias_presentes:
        df = df.assign(dia_1 = 0)
    if 2 not in dias_presentes:
        df = df.assign(dia_2 = 0)
    if 3 not in dias_presentes:
        df = df.assign(dia_3 = 0)
    if 4 not in dias_presentes:
        df = df.assign(dia_4 = 0)
    if 5 not in dias_presentes:
        df = df.assign(dia_5 = 0)
    if 6 not in dias_presentes:
        df = df.assign(dia_6 = 0)
    if 7 not in dias_presentes:
        df = df.assign(dia_7 = 0)
    df = df.assign(dia_sin = lambda x: np.sin(2 * np.pi * x.dia / 7))
    df = df.assign(dia_cos = lambda x: np.cos(2 * np.pi * x.dia / 7))
    
    df = pd.concat((df, pd.get_dummies(df.mes, prefix = 'mes')), axis = 1)
    meses_presentes = df.mes.unique()
    if 1 not in meses_presentes:
        df = df.assign(mes_1 = 0)
    if 2 not in meses_presentes:
        df = df.assign(mes_2 = 0)
    if 3 not in meses_presentes:
        df = df.assign(mes_3 = 0)
    if 4 not in meses_presentes:
        df = df.assign(mes_4 = 0)
    if 5 not in meses_presentes:
        df = df.assign(mes_5 = 0)
    if 6 not in meses_presentes:
        df = df.assign(mes_6 = 0)
    if 7 not in meses_presentes:
        df = df.assign(mes_7 = 0)
    if 8 not in meses_presentes:
        df = df.assign(mes_8 = 0)
    if 9 not in meses_presentes:
        df = df.assign(mes_9 = 0)
    if 10 not in meses_presentes:
        df = df.assign(mes_10 = 0)
    if 11 not in meses_presentes:
        df = df.assign(mes_11 = 0)
    if 12 not in meses_presentes:
        df = df.assign(mes_12 = 0)
    df = df.assign(mes_sin = lambda x: np.sin(2 * np.pi * x.mes / 12))
    df = df.assign(mes_cos = lambda x: np.cos(2 * np.pi * x.mes / 12))

    return df[['cliente',
               'anho',
               'dia',
               'dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7',
               'dia_sin','dia_cos',
               'mes',
               'mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12',
               'mes_sin','mes_cos',
               'prod']]

def retirar_data_validacion(df):
    # No hay manera de saber qué registros pertenecen a los días 10 al 14 de mayo, así que se va a considerar todo mayo como validación
    return df[~((df.anho == '2021') & (df.mes == 5))]

def corregir_dtypes_dataframe(df):
    df.cantidad = pd.to_numeric(df.cantidad)
    df.monto = pd.to_numeric(df.monto)
    df.descuento = pd.to_numeric(df.descuento)
    return df

def preparar_dataset(df):
    df = corregir_dtypes_dataframe(df)
    df = limpiar_dataset(df)
    df = seleccionar_clientes(df)
    df = generar_variables(df)
    df = retirar_data_validacion(df)
    df = df.drop(labels = ['anho'], axis = 1)
    return df

def seleccionar_clientes(df):
    #df_top_clientes = df[(df.cliente != 'SID1')] # & ~((df.anho == '2021') & (df.mes == 'Mayo'))
    #df_top_clientes = df_top_clientes[['cliente', 'monto']]
    #df_top_clientes = df_top_clientes.groupby(by = 'cliente', as_index = False).sum()
    #df_top_clientes = df_top_clientes.sort_values(by = 'monto', ascending = False)
    #df_top_clientes = df_top_clientes.head(round(len(df_top_clientes) * 0.2))
    #df_top_clientes

    meses = generar_diccionario_meses()
    df_frecuencia_clientes = df[(df.cliente != 'SID1')]
    df_frecuencia_clientes = df_frecuencia_clientes.assign(mes_numero = None)
    #df_frecuencia_clientes.mes_numero = df_frecuencia_clientes.mes.apply(lambda x: meses.get(x))
    df_frecuencia_clientes = df_frecuencia_clientes.assign(periodo = lambda x: pd.to_numeric(x.anho) * 100 + x.mes)
    df_frecuencia_clientes.periodo = df_frecuencia_clientes.periodo.apply(str)
    periodos = sorted(df_frecuencia_clientes.periodo.unique(), reverse = True)
    df_frecuencia_clientes = pd.pivot_table(df_frecuencia_clientes[['cliente','periodo','monto']], index = 'cliente', columns = 'periodo', values = 'monto', aggfunc = np.sum, fill_value = 0)
    df_frecuencia_clientes = df_frecuencia_clientes.assign(persistencia = '')
    for periodo_col in periodos:
        df_frecuencia_clientes.loc[df_frecuencia_clientes[periodo_col] > 0, periodo_col] = 1 # binarizamos
        df_frecuencia_clientes[periodo_col] = df_frecuencia_clientes[periodo_col].apply(lambda x: str(int(x)))
        df_frecuencia_clientes.loc[:,'persistencia'] = df_frecuencia_clientes.persistencia + df_frecuencia_clientes[periodo_col]

    df_frecuencia_clientes.sort_values(by = 'persistencia', ascending = False)
    df_frecuencia_clientes = df_frecuencia_clientes.assign(frecuencia_mensual_reciente = None)
    for p in range(len(periodos), 0, -1):
        df_frecuencia_clientes.frecuencia_mensual_reciente[df_frecuencia_clientes.frecuencia_mensual_reciente.isna()] = df_frecuencia_clientes[df_frecuencia_clientes.frecuencia_mensual_reciente.isna()].persistencia.apply(lambda x: p if x.startswith('1' * p) else None)

    return df[df.cliente.isin(df_frecuencia_clientes[df_frecuencia_clientes.frecuencia_mensual_reciente >= 3].index)]

def generar_variantes():
    variantes = {1: ['dia','prod'],
                 #2: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','prod'],
                 #3: ['dia_sin','dia_cos','prod'],
                 4: ['mes','prod'],
                 #5: ['mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 #6: ['mes_sin','mes_cos','prod'],
                 7: ['dia','mes','prod']}#,
                 #8: ['dia','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 #9: ['dia','mes_sin','mes_cos','prod'],
                 #10: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes','prod'],
                 #11: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 #12: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','mes_sin','mes_cos','prod'],
                 #13: ['dia_sin','dia_cos','mes','prod'],
                 #14: ['dia_sin','dia_cos','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','prod'],
                 #15: ['dia_sin','dia_cos','mes_sin','mes_cos','prod'],
                 #16: ['dia','dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','dia_sin','dia_cos','mes','mes_1','mes_2','mes_3','mes_4','mes_5','mes_6','mes_7','mes_8','mes_9','mes_10','mes_11','mes_12','mes_sin','mes_cos','prod']} # adición de último momento
    return variantes

def oversampling(X, y):
    if min(y.value_counts()) == 1:
        return RandomOverSampler().fit_resample(X, y)
    else:
        try:
            return SMOTE(k_neighbors = min(*y.value_counts(), 5)).fit_resample(X, y)
        except ValueError:
            return SMOTE(k_neighbors = min(*y.value_counts(), 5) - 1).fit_resample(X, y)
 
def aplicar_grid_search(X, y, modelo, params):
    if len(X) > 20:
        cv = StratifiedKFold(10)
    else:
        cv = LeaveOneOut()
    gs = GridSearchCV(estimator = modelo,
                      param_grid = params,
                      cv = cv,
                      scoring = ['accuracy','f1_weighted'],
                      refit = 'f1_weighted',
                      return_train_score = True,
                      verbose = 0,
                      n_jobs = -1)
    gs.fit(X, y)
    return (gs.best_estimator_,
            dict(test_f1_weighted = gs.best_score_,
                 train_f1_weighted = gs.cv_results_['mean_train_f1_weighted'][gs.best_index_],
                 test_accuracy = gs.cv_results_['mean_test_accuracy'][gs.best_index_],
                 train_accuracy = gs.cv_results_['mean_train_accuracy'][gs.best_index_]
                )
           )
 
def entrenar_LR(X, y):
    print('algoritmo LR')
    modelo = LogisticRegression(max_iter = 1000)
    params = dict(C = np.logspace(-6,2,9), solver = ('newton-cg','sag','saga','lbfgs'))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_SVM(X, y):
    print('algoritmo SVM')
    modelo = SVC()
    params = dict(C = np.logspace(-7,2,9), gamma = np.logspace(-7,2,9))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_RF(X, y):
    print('algoritmo RF')
    modelo = RandomForestClassifier(n_estimators = 70)
    params = dict(max_depth = range(3,11,2), min_samples_split = range(3,11,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_NB(X, y):
    print('algoritmo NB')
    modelo = GaussianNB()
    params = dict(var_smoothing = np.logspace(-11,-20, 10))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_KNN(X, y):
    print('algoritmo KNN')
    modelo = KNeighborsClassifier()
    params = dict(n_neighbors = range(2, 10, 3), weights = ('uniform','distance'), p = (1,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def busqueda_mejor_variante_algoritmo_hiperparametros(producto, df_producto, criterio_ordenacion):
    #print(df_producto['prod'].value_counts())
    df_producto.loc[df_producto['prod'] == producto, 'prod'] = 1
    df_producto.loc[df_producto['prod'] != 1, 'prod'] = 0
    X = df_producto.drop(labels = 'prod', axis = 1)
    y = df_producto['prod'].astype('int')
    X, y = oversampling(X, y)
    #print(y.value_counts())
    mejores_clasificadores_este_producto_y_cliente = list()
    for cod_variante, variante in sorted(generar_variantes().items(), reverse = False):
        print('cod_variante {}'.format(cod_variante))
        for entrenar_algoritmo in (entrenar_NB, entrenar_LR, entrenar_RF): #entrenar_SVM, entrenar_KNN, 
            mejor_clasificador, puntajes = entrenar_algoritmo(X[variante[:-1]], y)
            mejores_clasificadores_este_producto_y_cliente.append(dict(cod_variante = cod_variante, clasificador = mejor_clasificador, puntajes = puntajes))
    mejor_de_mejores = sorted(mejores_clasificadores_este_producto_y_cliente, key = criterio_ordenacion, reverse = True)[0]
    return mejor_de_mejores

def entrenar(df, variantes):
    global clasificadores_por_cliente
    criterio_ordenacion = lambda x: (x['puntajes']['test_f1_weighted'], x['puntajes']['test_accuracy'], x['puntajes']['train_f1_weighted'], x['puntajes']['train_accuracy'])
    lista_clientes = sorted(df.cliente.unique(), key = lambda cod_cli: int(cod_cli[3:]))
    clasificadores_por_cliente = dict()
    for cliente in lista_clientes:
        print('cliente {}'.format(cliente))
        df_cliente = df[df.cliente == cliente].drop(labels = 'cliente', axis = 1)
        lista_productos_este_cliente = sorted(df_cliente['prod'].unique(), key = lambda cod_prod: int(cod_prod[3:]))
        n_clases = len(lista_productos_este_cliente)
        clasificador_por_producto_este_cliente = dict()
        if n_clases == 1:
            continue
        elif n_clases == 2:
            producto_1, producto_2 = lista_productos_este_cliente
            mejor_de_mejores = busqueda_mejor_variante_algoritmo_hiperparametros(producto_2, df_cliente.copy(), criterio_ordenacion)
            mejor_de_mejores['clases_prod'] = {0: producto_1, 1: producto_2}
            clasificadores_por_cliente[cliente] = dict(n_clases = n_clases, solucion = mejor_de_mejores)
        elif n_clases > 2:
            for producto in lista_productos_este_cliente:
                print('producto {}'.format(producto))
                mejor_de_mejores = busqueda_mejor_variante_algoritmo_hiperparametros(producto, df_cliente.copy(), criterio_ordenacion)
                clasificador_por_producto_este_cliente[producto] = mejor_de_mejores
            clasificadores_por_cliente[cliente] = dict(n_clases = n_clases, solucion = clasificador_por_producto_este_cliente)
    return clasificadores_por_cliente

def main():
    df = crear_dataframe_desde_mongodb()
    df = preparar_dataset(df)
    variantes = generar_variantes()
    c = entrenar(df, variantes)
    return c

#clasificadores_por_cliente = None
rs = 10
np.random.seed(rs)
ans = main()

cliente SID13
producto KS_302
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_308
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_324
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_346
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_386
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_456
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo N

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_548
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_610
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_688
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_1009
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
cliente SID474
producto KS_362
cod_variante 1
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 4
algoritmo NB
algoritmo LR
algoritmo RF
cod_variante 7
algoritmo NB
algoritmo LR
algoritmo RF
producto KS_366
cod_variant

In [5]:
for k,v in clasificadores_por_cliente.items():
    print(k)
    print(v)

SID13
{'n_clases': 19, 'solucion': {'KS_302': {'cod_variante': 7, 'clasificador': GaussianNB(var_smoothing=1e-11), 'puntajes': {'test_f1_weighted': 0.9791666666666666, 'train_f1_weighted': 0.980377684295353, 'test_accuracy': 0.9800000000000001, 'train_accuracy': 0.9803870043000478}}, 'KS_308': {'cod_variante': 7, 'clasificador': GaussianNB(var_smoothing=1e-11), 'puntajes': {'test_f1_weighted': 0.9468309468309469, 'train_f1_weighted': 0.9508362248949309, 'test_accuracy': 0.95, 'train_accuracy': 0.9509675107501195}}, 'KS_324': {'cod_variante': 4, 'clasificador': GaussianNB(var_smoothing=1e-11), 'puntajes': {'test_f1_weighted': 0.9715617715617716, 'train_f1_weighted': 0.970576945297488, 'test_accuracy': 0.9718181818181819, 'train_accuracy': 0.9706043956043956}}, 'KS_346': {'cod_variante': 7, 'clasificador': GaussianNB(var_smoothing=1e-11), 'puntajes': {'test_f1_weighted': 0.9690656565656566, 'train_f1_weighted': 0.9699691022849102, 'test_accuracy': 0.97, 'train_accuracy': 0.97000000000000

In [6]:
import pickle
pickle_out = open('pickles/trabajo_v2.pkl', 'wb')
pickle.dump(clasificadores_por_cliente, pickle_out)
pickle_out.close()

In [3]:
import pickle
pickle_in = open('pickles/trabajo_v2.pkl', 'rb')
clasificadores_por_cliente = pickle.load(pickle_in)
pickle_in.close()

In [7]:
from sklearn.metrics import f1_score

df = crear_dataframe_desde_mongodb()
df = corregir_dtypes_dataframe(df)
df = limpiar_dataset(df)
df = seleccionar_clientes(df)

train = df[~((df.anho == '2021') & (df.mes == 5))]
train = generar_variables(train)

test = df[((df.anho == '2021') & (df.mes == 5))]
test = generar_variables(test)

clasificadores = clasificadores_por_cliente
variantes = generar_variantes()

curvas_por_cliente = dict()
#for cliente, (cod_variante, clasificador) in clasificadores.items():
for cliente, values in clasificadores.items():
    #if cliente != 'SID339': continue
    print(cliente)
    train_cliente = train[train.cliente == cliente]
    test_cliente = test[test.cliente == cliente]
    curvas_por_producto = dict()
    for producto, solucion in values['solucion'].items():
        if producto not in test_cliente['prod'].unique(): continue
        print(producto)
        
        train_cliente_producto = train_cliente.copy()
        train_cliente_producto.loc[train_cliente_producto['prod'] == producto, 'prod'] = 1
        train_cliente_producto.loc[train_cliente_producto['prod'] != 1, 'prod'] = 0
        
        test_cliente_producto = test_cliente.copy()
        test_cliente_producto.loc[test_cliente_producto['prod'] == producto, 'prod'] = 1
        test_cliente_producto.loc[test_cliente_producto['prod'] != 1, 'prod'] = 0
        
        cod_variante = solucion['cod_variante']
        clasificador = solucion['clasificador']
        puntajes_cv = solucion['puntajes']
        
        train_cliente_producto = train_cliente_producto[variantes[cod_variante]]
        test_cliente_producto = test_cliente_producto[variantes[cod_variante]]
        
        X_train_cliente_producto = train_cliente_producto.drop(labels = 'prod', axis = 1)
        y_train_cliente_producto = train_cliente_producto['prod'].astype('int')
        X_train_cliente_producto, y_train_cliente_producto = oversampling(X_train_cliente_producto, y_train_cliente_producto)
        train_oversampled = X_train_cliente_producto.join(y_train_cliente_producto)
        train_oversampled_producto_0 = train_oversampled[train_oversampled['prod'] == 0]
        train_oversampled_producto_1 = train_oversampled[train_oversampled['prod'] == 1]
        cantidad_producto_0 = len(train_oversampled_producto_0)
        cantidad_producto_1 = len(train_oversampled_producto_1)
        for i in range(min(cantidad_producto_0, cantidad_producto_1)):
            if i == 0:
                train_resorted = train_oversampled_producto_0.iloc[[i]]
                train_resorted = train_resorted.append(train_oversampled_producto_1.iloc[i])
            else:
                train_resorted = train_resorted.append(train_oversampled_producto_0.iloc[i])
                train_resorted = train_resorted.append(train_oversampled_producto_1.iloc[i])

            if (i+1) == min(cantidad_producto_0, cantidad_producto_1):
                producto_mas_grande = np.argmax([cantidad_producto_0, cantidad_producto_1])
                if producto_mas_grande == 0:
                    train_resorted = train_resorted.append(train_oversampled_producto_0.iloc[i+1:])
                elif producto_mas_grande == 1:
                    train_resorted = train_resorted.append(train_oversampled_producto_1.iloc[i+1:])
                break
                
        X_train_cliente_producto = train_resorted.drop(labels = 'prod', axis = 1)
        y_train_cliente_producto = train_resorted['prod'].astype('int')
            
        X_test_cliente_producto = test_cliente_producto.drop(labels = 'prod', axis = 1)
        y_test_cliente_producto = test_cliente_producto['prod'].astype('int')
        #X_test_cliente_producto_os, y_test_cliente_producto_os = oversampling(X_test_cliente_producto_og, y_test_cliente_producto_og)

        cantidad_training_samples = len(X_train_cliente_producto)
        curva_aprendizaje = dict()
        for train_samples in range(cantidad_training_samples):
            train_samples += 1
            if train_samples == 1: continue
            clasificador.fit(X_train_cliente_producto.head(train_samples), y_train_cliente_producto.head(train_samples))
            try:
                y_train_pred = clasificador.predict(X_train_cliente_producto)
            except ValueError as e:
                if type(clasificador) == KNeighborsClassifier:
                    continue
                else:
                    raise e
            y_test_pred = clasificador.predict(X_test_cliente_producto)
            #y_test_pred_os = clasificador.predict(X_test_cliente_producto_os)
            score_train = f1_score(y_train_cliente_producto, y_train_pred, average = 'weighted')
            score_test = f1_score(y_test_cliente_producto, y_test_pred, average = 'weighted')
            #score_test_os = f1_score(y_test_cliente_producto_os, y_test_pred_os, average = 'weighted')
            
            curva_aprendizaje[train_samples] = dict(score_train = score_train, score_test = score_test) #, score_test_os = score_test_os)
            
        curvas_por_producto[producto] = curva_aprendizaje
    curvas_por_cliente[cliente] = curvas_por_producto

SID13
KS_536
SID23
SID30
KS_369
SID55
KS_415
KS_673
KS_675
SID127
KS_386


/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/

KS_414
KS_446
KS_484
KS_688
SID170
KS_644
SID265
KS_308
KS_324
KS_336
KS_346
KS_348
KS_354
KS_364
KS_476
KS_538
KS_582
KS_624
KS_688
KS_950
KS_971
KS_1024
KS_1033
SID300
SID315
KS_369
SID320
KS_653


/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/

SID324
SID339
KS_324


/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/

KS_652
KS_654
SID340
KS_346
KS_348
KS_360
SID357
SID467
KS_548
SID474
KS_362
KS_366
KS_950


/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/

SID511
SID517
KS_367
KS_939


/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/opt/conda/lib/python3.7/site-packages/sklearn/naive_bayes.py:454: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/opt/conda/lib/python3.7/site-packages/

In [9]:
pickle_out = open('pickles/validacion_v2.pkl', 'wb')
pickle.dump(curvas_por_cliente, pickle_out)
pickle_out.close()

In [ ]:
curvas_por_cliente

In [ ]:
validacion